In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
    !pwd

/home/aditya/Downloads/MACHINE-LEARNING/self-study


In [3]:
!ls

basic-analysis-iris.ipynb		       kaggle.ipynb
complete.csv				       sample-submission.csv
kaggle-community-olympiad-unmasking-fakes.zip  validate.csv


In [4]:
# Load the dataset
df = pd.read_csv('complete.csv')

# Display basic information and first few rows
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4887 entries, 0 to 4886
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           4887 non-null   int64 
 1   Label                        4887 non-null   bool  
 2   Statement                    4887 non-null   object
 3   Subject                      4887 non-null   object
 4   Speaker                      4887 non-null   object
 5   Speaker Job Title            3503 non-null   object
 6   State                        3827 non-null   object
 7   Political Party Affiliation  4887 non-null   object
 8   True Count                   4887 non-null   int64 
 9   False Count                  4887 non-null   int64 
 10  Context                      4833 non-null   object
dtypes: bool(1), int64(3), object(7)
memory usage: 386.7+ KB


(None,
       ID  Label                                          Statement  \
 0   3115   True  The United States has one of the highest corpo...   
 1   7925  False  The Foreign Intelligence Surveillance Court is...   
 2  12380   True  Half of illegal immigrants come on legal visas...   
 3   2439  False  Sakineh Mohammadi Ashtiani was never sentenced...   
 4  10054  False  At least 10 ISIS fighters have been caught com...   
 
                                Subject              Speaker Speaker Job Title  \
 0           corporations,economy,taxes         barack-obama         President   
 1  civil-rights,legal-issues,terrorism         barack-obama         President   
 2                          immigration       ken-cuccinelli  Attorney General   
 3     crime,human-rights,abc-news-week  mahmoud-ahmadinejad               NaN   
 4                            terrorism      duncan-hunter-2       Congressman   
 
         State Political Party Affiliation  True Count  False Count  \


In [5]:
df.nunique()

ID                             4887
Label                             2
Statement                      4882
Subject                        2121
Speaker                        1760
Speaker Job Title               727
State                            62
Political Party Affiliation      19
True Count                       27
False Count                      46
Context                        2422
dtype: int64

In [6]:
df = df.drop(columns=['ID'])

In [7]:
df

,Label,Statement,Subject,Speaker,Speaker Job Title,State,Political Party Affiliation,True Count,False Count,Context
0,True,The United States has one of the highest corpo...,"corporations,economy,taxes",barack-obama,President,Illinois,democrat,163,150,the State of the Union address
1,False,The Foreign Intelligence Surveillance Court is...,"civil-rights,legal-issues,terrorism",barack-obama,President,Illinois,democrat,163,150,an interview with Charlie Rose
2,True,Half of illegal immigrants come on legal visas...,immigration,ken-cuccinelli,Attorney General,Virginia,republican,2,12,a radio interview.
3,False,Sakineh Mohammadi Ashtiani was never sentenced...,"crime,human-rights,abc-news-week",mahmoud-ahmadinejad,NaN,NaN,none,0,2,"ABC's ""This Week"""
4,False,At least 10 ISIS fighters have been caught com...,terrorism,duncan-hunter-2,Congressman,California,republican,0,1,an interview with Greta Van Susteren of Fox News
...,...,...,...,...,...,...,...,...,...,...
4882,True,Says Mitt Romney had millions in a Swiss bank ...,"candidates-biography,message-machine-2012",barack-obama,President,Illinois,democrat,163,150,a campaign ad
4883,False,"Between 1990 and 2013, the maternal mortality ...",women,sheila-jackson-lee,congresswoman,Texas,democrat,0,3,remarks inserted into the Congressional Record
4884,False,Says Apple products are currently 90 percent c...,"corporations,human-rights,technology",robin-wright,NaN,NaN,newsmaker,0,1,an event at the Rockefeller Center
4885,False,We have more natural gas than Saudi Arabia has...,"energy,environment,transportation",tommy-thompson,NaN,Wisconsin,republican,5,17,a speech


In [8]:
# Fill missing categorical values with 'Unknown'
categorical_cols = ['Speaker Job Title', 'State', 'Context']
df[categorical_cols] = df[categorical_cols].fillna('Unknown')

In [9]:
# Encode categorical columns with LabelEncoder (for high-cardinality features)
label_enc = LabelEncoder()
df['Speaker'] = label_enc.fit_transform(df['Speaker'])
df['Subject'] = label_enc.fit_transform(df['Subject'])
df['Political Party Affiliation'] = label_enc.fit_transform(df['Political Party Affiliation'])
df['State'] = label_enc.fit_transform(df['State'])

In [10]:
df

,Label,Statement,Subject,Speaker,Speaker Job Title,State,Political Party Affiliation,True Count,False Count,Context
0,True,The United States has one of the highest corpo...,688,123,President,13,4,163,150,the State of the Union address
1,False,The Foreign Intelligence Surveillance Court is...,566,123,President,13,4,163,150,an interview with Charlie Rose
2,True,Half of illegal immigrants come on legal visas...,1788,951,Attorney General,52,15,2,12,a radio interview.
3,False,Sakineh Mohammadi Ashtiani was never sentenced...,859,1052,Unknown,47,13,0,2,"ABC's ""This Week"""
4,False,At least 10 ISIS fighters have been caught com...,2099,483,Congressman,5,15,0,1,an interview with Greta Van Susteren of Fox News
...,...,...,...,...,...,...,...,...,...,...
4882,True,Says Mitt Romney had millions in a Swiss bank ...,355,123,President,13,4,163,150,a campaign ad
4883,False,"Between 1990 and 2013, the maternal mortality ...",2118,1535,congresswoman,45,4,0,3,remarks inserted into the Congressional Record
4884,False,Says Apple products are currently 90 percent c...,700,1453,Unknown,47,12,0,1,an event at the Rockefeller Center
4885,False,We have more natural gas than Saudi Arabia has...,1365,1677,Unknown,59,15,5,17,a speech


In [11]:
df = pd.get_dummies(df, columns=['Political Party Affiliation', 'State'])

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Speaker'] = le.fit_transform(df['Speaker'])
df['Subject'] = le.fit_transform(df['Subject'])

In [13]:
df

,Label,Statement,Subject,Speaker,Speaker Job Title,True Count,False Count,Context,Political Party Affiliation_0,Political Party Affiliation_1,...,State_53,State_54,State_55,State_56,State_57,State_58,State_59,State_60,State_61,State_62
0,True,The United States has one of the highest corpo...,688,123,President,163,150,the State of the Union address,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,The Foreign Intelligence Surveillance Court is...,566,123,President,163,150,an interview with Charlie Rose,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,Half of illegal immigrants come on legal visas...,1788,951,Attorney General,2,12,a radio interview.,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,Sakineh Mohammadi Ashtiani was never sentenced...,859,1052,Unknown,0,2,"ABC's ""This Week""",False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,At least 10 ISIS fighters have been caught com...,2099,483,Congressman,0,1,an interview with Greta Van Susteren of Fox News,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,True,Says Mitt Romney had millions in a Swiss bank ...,355,123,President,163,150,a campaign ad,False,False,...,False,False,False,False,False,False,False,False,False,False
4883,False,"Between 1990 and 2013, the maternal mortality ...",2118,1535,congresswoman,0,3,remarks inserted into the Congressional Record,False,False,...,False,False,False,False,False,False,False,False,False,False
4884,False,Says Apple products are currently 90 percent c...,700,1453,Unknown,0,1,an event at the Rockefeller Center,False,False,...,False,False,False,False,False,False,False,False,False,False
4885,False,We have more natural gas than Saudi Arabia has...,1365,1677,Unknown,5,17,a speech,False,False,...,False,False,False,False,False,False,True,False,False,False


In [14]:
# Use TF-IDF Vectorization for text processing
vectorizer = TfidfVectorizer(max_features=5000)  # Limit vocabulary to avoid overfitting
X_text = vectorizer.fit_transform(df['Statement']).toarray()

In [15]:
df

,Label,Statement,Subject,Speaker,Speaker Job Title,True Count,False Count,Context,Political Party Affiliation_0,Political Party Affiliation_1,...,State_53,State_54,State_55,State_56,State_57,State_58,State_59,State_60,State_61,State_62
0,True,The United States has one of the highest corpo...,688,123,President,163,150,the State of the Union address,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,The Foreign Intelligence Surveillance Court is...,566,123,President,163,150,an interview with Charlie Rose,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,Half of illegal immigrants come on legal visas...,1788,951,Attorney General,2,12,a radio interview.,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,Sakineh Mohammadi Ashtiani was never sentenced...,859,1052,Unknown,0,2,"ABC's ""This Week""",False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,At least 10 ISIS fighters have been caught com...,2099,483,Congressman,0,1,an interview with Greta Van Susteren of Fox News,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,True,Says Mitt Romney had millions in a Swiss bank ...,355,123,President,163,150,a campaign ad,False,False,...,False,False,False,False,False,False,False,False,False,False
4883,False,"Between 1990 and 2013, the maternal mortality ...",2118,1535,congresswoman,0,3,remarks inserted into the Congressional Record,False,False,...,False,False,False,False,False,False,False,False,False,False
4884,False,Says Apple products are currently 90 percent c...,700,1453,Unknown,0,1,an event at the Rockefeller Center,False,False,...,False,False,False,False,False,False,False,False,False,False
4885,False,We have more natural gas than Saudi Arabia has...,1365,1677,Unknown,5,17,a speech,False,False,...,False,False,False,False,False,False,True,False,False,False


In [19]:
X_numeric = df[['True Count', 'False Count'] + 
               [col for col in df.columns if col.startswith('Political Party Affiliation_')] + 
               [col for col in df.columns if col.startswith('State_')]]


In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)  # Scale numerical features


In [20]:
print(df.columns)

Index(['Label', 'Statement', 'Subject', 'Speaker', 'Speaker Job Title',
       'True Count', 'False Count', 'Context', 'Political Party Affiliation_0',
       'Political Party Affiliation_1', 'Political Party Affiliation_2',
       'Political Party Affiliation_3', 'Political Party Affiliation_4',
       'Political Party Affiliation_5', 'Political Party Affiliation_6',
       'Political Party Affiliation_7', 'Political Party Affiliation_8',
       'Political Party Affiliation_9', 'Political Party Affiliation_10',
       'Political Party Affiliation_11', 'Political Party Affiliation_12',
       'Political Party Affiliation_13', 'Political Party Affiliation_14',
       'Political Party Affiliation_15', 'Political Party Affiliation_16',
       'Political Party Affiliation_17', 'Political Party Affiliation_18',
       'State_0', 'State_1', 'State_2', 'State_3', 'State_4', 'State_5',
       'State_6', 'State_7', 'State_8', 'State_9', 'State_10', 'State_11',
       'State_12', 'State_13', 'St

In [23]:
from sklearn.model_selection import train_test_split

# Define the target variable (Label)
y = df['Label']  # Assuming 'Label' is the target column

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X_numeric_scaled, y, test_size=0.2, random_state=42)

# Check the shape
print("Training set:", X_train.shape, y_train.shape)
print("Testing set:", X_test.shape, y_test.shape)


Training set: (3909, 84) (3909,)
Testing set: (978, 84) (978,)


In [24]:
model = LogisticRegression(max_iter=1000)  # Increase from default 100
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [25]:
# Predictions
y_pred = model.predict(X_test)

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Classification Report
print(classification_report(y_test, y_pred))

Accuracy: 0.63
              precision    recall  f1-score   support

       False       0.65      0.75      0.70       564
        True       0.58      0.46      0.51       414

    accuracy                           0.63       978
   macro avg       0.62      0.61      0.61       978
weighted avg       0.62      0.63      0.62       978



In [ ]:
# Correcting OneHotEncoder parameter
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical = encoder.fit_transform(df[categorical_cols])

# TF-IDF Vectorization for text data (Statement column)
tfidf = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features
text_features = tfidf.fit_transform(df["Statement"])

# Normalize numeric columns (True Count, False Count)
scaler = StandardScaler()
numeric_features = scaler.fit_transform(df[["True Count", "False Count"]])

# Final Feature Matrix
X = np.hstack((text_features.toarray(), encoded_categorical, numeric_features))

# Target Variable
y = df["Label"].astype(int)

# Check final shape
X.shape, y.shape


In [ ]:
from sklearn.model_selection import train_test_split

# Splitting dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check the shape of the split datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Check training completion
model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display the results
accuracy, precision, recall, f1


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

# Calculate evaluation metrics for Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# Display results
accuracy_rf, precision_rf, recall_rf, f1_rf


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize Random Forest model
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Use RandomizedSearchCV to find the best parameters
rf_search = RandomizedSearchCV(rf, param_grid, n_iter=10, cv=3, scoring='f1', n_jobs=-1, random_state=42)
rf_search.fit(X_train, y_train)

# Get the best model
best_rf_model = rf_search.best_estimator_

# Predict on test set
y_pred_best_rf = best_rf_model.predict(X_test)

# Evaluate the tuned model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
precision_best_rf = precision_score(y_test, y_pred_best_rf)
recall_best_rf = recall_score(y_test, y_pred_best_rf)
f1_best_rf = f1_score(y_test, y_pred_best_rf)

# Display results
rf_search.best_params_, (accuracy_best_rf, precision_best_rf, recall_best_rf, f1_best_rf)


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
df = pd.read_csv('complete.csv')

# Drop rows with missing values in critical text columns
df = df.dropna(subset=["Statement"])

# Convert Label (True/False) to binary values (True -> 1, False -> 0)
df["Label"] = df["Label"].astype(int)

# Text Preprocessing Function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply text cleaning
df["Cleaned_Statement"] = df["Statement"].apply(clean_text)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(df["Cleaned_Statement"], df["Label"], test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train XGBoost Model with NLP features
model = XGBClassifier(n_estimators=300, max_depth=50, eval_metric='logloss')
xgb_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred_xgb = xgb_model.predict(X_test_tfidf)

# Evaluate Performance
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

# Display Results
(accuracy_xgb, precision_xgb, recall_xgb, f1_xgb)
